In [19]:
import os
import ipykernel
import pathlib

# Read the file and load the keys
token_file = open('token.txt', 'r')
# Read the first line
open_ai_token_line = token_file.readline()

if open_ai_token_line:
    open_ai_key = open_ai_token_line.split('=')[1].strip()
    print(f'Open AI Key  is loaded')

# Read the second line
langsmith_token_line = token_file.readline()

if langsmith_token_line:
    langsmith_key = langsmith_token_line.split('=')[1].strip()
    print(f'Langsmith Key is loaded')


if langsmith_key is not None:
    # Configure LangSmith
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "MULTI-AGENT"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = langsmith_key

Open AI Key  is loaded
Langsmith Key is loaded


### Create agents

In [20]:
import json

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    functions = [format_tool_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards defining the key elements to create a View. \
                    The final response should be a JSON text that includes filters, join rules and the potential comparisons."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

### Define tools

In [10]:
from langchain_core.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# Warning: This executes code locally, which can be unsafe when not sandboxed

repl = PythonREPL()

@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed:\n```python\n{code}\n```\nStdout: {result}"

@tool
def format_filter(
    model_identifier: Annotated[str, "The identifier of the model to use for the filter."],
    class_name: Annotated[str, "The class name of the model to use for the filter."],
):
    return f"{{model_identifier}}.class_name.*"

#### Agent State

In [11]:
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

#### Agent Nodes

In [13]:
import functools

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }


llm = ChatOpenAI(model="gpt-3.5-turbo-0613", openai_api_key=open_ai_key)

# Research agent and node
research_agent = create_agent(
    llm, 
    [search], 
    system_message="You should provide accurate data for the chart generator to use.",
)
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Chart Generator
chart_agent = create_agent(
    llm,
    [python_repl],
    system_message="Any charts you display will be visible by the user.",
)
chart_node = functools.partial(agent_node, agent=chart_agent, name="Chart Generator")

#### Tool node

In [14]:
tools = [search, python_repl]
tool_executor = ToolExecutor(tools)

def tool_node(state):
    """This runs tools in the graph

    It takes in an agent action and calls that tool and returns the result."""
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    tool_input = json.loads(
        last_message.additional_kwargs["function_call"]["arguments"]
    )
    # We can pass single-arg inputs by value
    if len(tool_input) == 1 and "__arg1" in tool_input:
        tool_input = next(iter(tool_input.values()))
    tool_name = last_message.additional_kwargs["function_call"]["name"]
    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_input,
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(
        content=f"{tool_name} response: {str(response)}", name=action.tool
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

#### Edge

In [15]:
# Either agent can decide to end
def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "function_call" in last_message.additional_kwargs:
        # The previus agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "end"
    return "continue"

In [16]:
workflow = StateGraph(AgentState)

workflow.add_node("Researcher", research_node)
workflow.add_node("Chart Generator", chart_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "Researcher",
    router,
    {"continue": "Chart Generator", "call_tool": "call_tool", "end": END},
)
workflow.add_conditional_edges(
    "Chart Generator",
    router,
    {"continue": "Researcher", "call_tool": "call_tool", "end": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "Researcher": "Researcher",
        "Chart Generator": "Chart Generator",
    },
)
workflow.set_entry_point("Researcher")
graph = workflow.compile()

In [18]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Fetch the UK's GDP over the past 5 years,"
                " then draw a line graph of it."
                " Once you code it up, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 100},
):
    print(s)
    print("----")

{'Researcher': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "UK GDP over the past 5 years"\n}', 'name': 'duckduckgo_search'}}, name='Researcher')], 'sender': 'Researcher'}}
----
{'call_tool': {'messages': [FunctionMessage(content="duckduckgo_search response: Economy Gross Domestic Product (GDP) Gross Domestic Product: Year on Year growth: CVM SA % Gross Domestic Product: Year on Year growth: CVM SA % Source dataset: GDP first quarterly... Q3 2023: 568,953 (+ more) Updated: Jan 8, 2024 Units: Domestic Currency, Seasonally Adjusted Frequency: Quarterly 1Y | 5Y | 10Y | Max to Edit Graph Share Links Account Tools NOTES Source: International Monetary Fund Release: International Financial Statistics Units: Domestic Currency, Seasonally Adjusted Frequency: Quarterly Suggested Citation: An annual GDP growth rate of 7.6 percent was recorded in 2021, and the unemployment rate has quickly recovered to its pre-pandemic levels of less than 

Python REPL can execute arbitrary code. Use with caution.


{'Chart Generator': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "code": "import matplotlib.pyplot as plt\\n\\nyears = [2018, 2019, 2020, 2021, 2022]\\ngdp_growth = [1.4, 1.5, -9.4, 8.7, 4.3]\\n\\nplt.plot(years, gdp_growth, marker=\'o\')\\nplt.xlabel(\'Year\')\\nplt.ylabel(\'GDP Growth Rate (%)\')\\nplt.title(\\"UK\'s GDP Growth Over the Past 5 Years\\")\\nplt.grid(True)\\nplt.show()"\n}', 'name': 'python_repl'}}, name='Chart Generator')], 'sender': 'Chart Generator'}}
----
{'call_tool': {'messages': [FunctionMessage(content='python_repl response: Succesfully executed:\n```python\nimport matplotlib.pyplot as plt\n\nyears = [2018, 2019, 2020, 2021, 2022]\ngdp_growth = [1.4, 1.5, -9.4, 8.7, 4.3]\n\nplt.plot(years, gdp_growth, marker=\'o\')\nplt.xlabel(\'Year\')\nplt.ylabel(\'GDP Growth Rate (%)\')\nplt.title("UK\'s GDP Growth Over the Past 5 Years")\nplt.grid(True)\nplt.show()\n```\nStdout: ModuleNotFoundError("No module named \'matplotl